In [0]:
dbutils.fs.cp("file:/Workspace/Shared/Sales_data3.csv", "dbfs:/FileStore/streaming/input/sales_data.csv")
dbutils.fs.cp("file:/Workspace/Shared/customer_data1.json", "dbfs:/FileStore/streaming/input/customer_data.json")
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("StructuredStreamingExample") \
    .getOrCreate()

# Define the schema for the CSV data
sales_schema = "OrderID INT, OrderDate STRING, CustomerID STRING, Product STRING, Quantity INT, Price DOUBLE"

# Read streaming data from CSV files
df_sales_stream = spark.readStream \
    .format("csv") \
    .option("header", "true") \
    .schema(sales_schema) \
    .load("dbfs:/FileStore/streaming/input/")

# Define the schema for the JSON data
customer_schema = "CustomerID STRING, CustomerName STRING, Region STRING, SignupDate STRING"

# Read streaming data from JSON files
df_customers_stream = spark.readStream \
    .format("json") \
    .schema(customer_schema) \
    .load("dbfs:/FileStore/streaming/input/")

df_customers_stream.printSchema()

root
 |-- CustomerID: string (nullable = true)
 |-- CustomerName: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- SignupDate: string (nullable = true)



In [0]:
from pyspark.sql.functions import current_date, datediff, to_timestamp

# Transform the sales data: Add a new column for total amount
df_sales_transformed = df_sales_stream.select(
    col("OrderID"),
    to_timestamp(col("OrderDate"), "yyyy-MM-dd HH:mm:ss").alias("OrderDate"), # Convert OrderDate to TIMESTAMP
    col("Product"),
    col("Quantity"),
    col("Price"),
    (col("Quantity") * col("Price")).alias("TotalAmount")
)

print("Applied transformations on sales data...")

# Add watermark to handle late data and perform an aggregation
df_sales_aggregated = df_sales_transformed \
    .withWatermark("OrderDate", "10 days") \
    .groupBy("Product") \
    .agg({"TotalAmount": "sum"})

print("Aggregated sales data by product...")

# Transform the customer data: Add a new column for the number of years since signup
df_customers_transformed = df_customers_stream.withColumn(
    "YearsSinceSignup",
    datediff(current_date(), to_timestamp(col("SignupDate"), "yyyy-MM-dd")).cast("int") / 365
)

print("Applied transformations on customer data...")
# Write the aggregated sales data to a console sink for debugging
sales_query = df_sales_aggregated.writeStream \
    .outputMode("update") \
    .format("console") \
    .start()

print("Started streaming query to write aggregated sales data to console...")

# Write the transformed customer data to a console sink for debugging
customers_query = df_customers_transformed.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

print("Started streaming query to write transformed customer data to console...")

Applied transformations on sales data...
Aggregated sales data by product...
Applied transformations on customer data...
Started streaming query to write aggregated sales data to console...
Started streaming query to write transformed customer data to console...


In [0]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp

# Create sample sales data
data = {
    "OrderID": [1, 2, 3, 4],
    "OrderDate": ["2024-01-01 10:00:00", "2024-01-02 11:00:00", "2024-01-03 12:00:00", "2024-01-04 13:00:00"],
    "CustomerID": ["C001", "C002", "C003", "C004"],
    "Product": ["ProductA", "ProductB", "ProductC", "ProductD"],
    "Quantity": [10, 20, 15, 5],
    "Price": [100.0, 200.0, 150.0, 50.0]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Save to CSV
csv_path = "/dbfs/FileStore/sales_data.csv"
df.to_csv(csv_path, index=False)

print(f"Sample data saved to {csv_path}")

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("StructuredStreamingExample") \
    .getOrCreate()

# Load data from CSV
df = spark.read.format("csv").option("header", "true").load("/FileStore/sales_data.csv")

print("Data Loaded Successfully")

# Transform the data: Add a new column for total amount
df_transformed = df.withColumn("TotalAmount", col("Quantity").cast("int") * col("Price").cast("double"))

print("Data Transformed Successfully")

# Write transformed data to a Delta table
df_transformed.write.format("delta").mode("overwrite").save("/delta/sales_data")

print("Transformed data written to Delta table successfully")

Sample data saved to /dbfs/FileStore/sales_data.csv
Data Loaded Successfully
Data Transformed Successfully
Transformed data written to Delta table successfully


In [0]:
import pandas as pd

# Create sample sales data
sales_data = {
    "OrderID": [1, 2, 3, 4],
    "OrderDate": ["2024-01-01 10:00:00", "2024-01-02 11:00:00", "2024-01-03 12:00:00", "2024-01-04 13:00:00"],
    "CustomerID": ["C001", "C002", "C003", "C004"],
    "Product": ["ProductA", "ProductB", "ProductC", "ProductD"],
    "Quantity": [10, 20, 15, 5],
    "Price": [100.0, 200.0, 150.0, 50.0]
}

# Convert to DataFrame
df_sales = pd.DataFrame(sales_data)

# Save as CSV
csv_path = "/dbfs/FileStore/sales_data.csv"
df_sales.to_csv(csv_path, index=False)

# Save as Parquet
parquet_path = "/dbfs/FileStore/sales_data.parquet"
df_sales.to_parquet(parquet_path, index=False)

print(f"Sample data saved to {csv_path} and {parquet_path}")

Sample data saved to /dbfs/FileStore/sales_data.csv and /dbfs/FileStore/sales_data.parquet


In [0]:
# Initialize SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp

spark = SparkSession.builder \
    .appName("DeltaExample") \
    .getOrCreate()

# Load data from CSV
df_sales = spark.read.format("csv").option("header", "true").load("/FileStore/sales_data.csv")

# Transform the data: Add a new column for total amount
df_transformed = df_sales.withColumn("TotalAmount", col("Quantity").cast("int") * col("Price").cast("double"))

# Write transformed data to Delta table
delta_table_path = "/delta/sales_data"
df_transformed.write.format("delta").mode("overwrite").save(delta_table_path)

print("Delta table created and data written successfully.")

Delta table created and data written successfully.


In [0]:
import dlt

@dlt.table
def sales_data():
    df = spark.read.format("delta").load(delta_table_path)
    return df.select(
        col("OrderID"),
        col("OrderDate"),
        col("CustomerID"),
        col("Product"),
        col("Quantity"),
        col("Price"),
        (col("Quantity").cast("int") * col("Price").cast("double")).alias("TotalAmount")
    )

print("Delta Live Table created.")

Delta Live Table created.


Name,Type
OrderID,string
OrderDate,string
CustomerID,string
Product,string
Quantity,string
Price,string
TotalAmount,double


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("DeltaOperationsSimpleExample") \
    .getOrCreate()

# Define Delta table path
delta_table_path = "/delta/simple_data"

# Define initial sample data
initial_data = [
    (1, 100),
    (2, 200),
    (3, 300)
]

# Define schema
schema = ["ID", "Value"]

# Create DataFrame for initial data
df_initial = spark.createDataFrame(initial_data, schema=schema)

# Write DataFrame to Delta table
df_initial.write.format("delta").mode("overwrite").save(delta_table_path)

print("Initial Delta table created and data written successfully.")

# Define new sample data
new_sample_data = [
    (2, 250),  # Existing ID with updated Value
    (4, 400)   # New ID
]

# Create DataFrame for new data
df_new = spark.createDataFrame(new_sample_data, schema=schema)

# Write the new data to Delta table in append mode
df_new.write.format("delta").mode("append").save(delta_table_path)

print("New data appended to Delta table successfully.")

# Create a temporary view for SQL operations
df_new.createOrReplaceTempView("new_data")

# Perform the merge operation
print("Merging new data into Delta table...")

spark.sql(f"""
MERGE INTO delta.`{delta_table_path}` AS target
USING new_data AS source
ON target.ID = source.ID
WHEN MATCHED THEN UPDATE SET
    target.Value = source.Value
WHEN NOT MATCHED THEN INSERT (
    ID,
    Value
) VALUES (
    source.ID,
    source.Value
)
""")

print("Data merged successfully.")

# Delta operations - History, Time Travel, and Vacuum
print("Viewing Delta table history...")
history_df = spark.sql(f"DESCRIBE HISTORY delta.`{delta_table_path}`")
history_df.show(truncate=False)

print("Querying Delta table as of version 0...")
df_time_travel = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
df_time_travel.show(truncate=False)

print("Vacuuming old files...")
spark.sql(f"VACUUM delta.`{delta_table_path}` RETAIN 168 HOURS")

print("Delta operations completed.")

Initial Delta table created and data written successfully.
New data appended to Delta table successfully.
Merging new data into Delta table...
Data merged successfully.
Viewing Delta table history...
+-------+-------------------+----------------+----------------------------------+---------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+------------------+--------------------+-----------+-----------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------